In [2]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sys

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import glob

import albumentations as A
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import roc_auc_score
from torch.optim import lr_scheduler
from tqdm import tqdm
import re
import random
import cv2
from torch.utils.data import Dataset
import monai
import matplotlib as plt

sys.path.append("../../RSNA-Brain-Tumor-Classification/utils")
from classifier_utils import *

IMAGE_SIZE = 256

In [3]:
import nibabel as nib
from dataset_utils import *

def isometric_numpy_transform(sample, scale_factors, img_size):
    #print(scale_factors)
    #print(sample.shape)
    # With Tensor3D Transformation, idx 0 becomes 2, idx 1 becomes 0 and idx 2 becomes 1
    sample = sample - np.min(sample)
    sample = sample[:,:,:,np.newaxis]
    s1 = int(sample.shape[0]*scale_factors[0])
    s2 = int(sample.shape[1]*scale_factors[1])
    s3 = int(sample.shape[2]*scale_factors[2])
    transform = transforms.Compose([ToTensor3D(),
                                    Resize3D(size=(s3,s1,s2)),
                                    Rotation3D90Degrees(times=2, axis=1),
                                    Rotation3D90Degrees(times=2, axis=3),
                                    Pad3D(size=(img_size,img_size,img_size))
    ])
    sample = transform(sample)
    sample = sample.squeeze(0).numpy()
    return sample

def load_nifti_image(path, img_size=IMAGE_SIZE):
    nifti = nib.load(path)
    data = nifti.get_fdata().astype(np.float32)
    scale_factors = nifti.header.get_zooms()
    data = isometric_numpy_transform(data, scale_factors, img_size)
    
    return data

In [4]:
from scipy import io
import os
from math import floor, ceil

def save_dicom_images_3d_as_mat(
    folder,
    mri_type,
    case_id,
    target,
    img_size=IMAGE_SIZE,
    #rotate=2,
):
    case_id = str(case_id).zfill(5)
    
    if folder == "test":
        target = "unknown"

    path = f"../../RSNA-BTC-Datasets/{folder}/{case_id}/{mri_type}/*.dcm"
    nii_path = f"../../RSNA-BTC-Datasets/{folder}_nii/{mri_type}/{target}/{case_id}.nii.gz"
    files = sorted(
        glob.glob(path),
        key=lambda var: [
            int(x) if x.isdigit() else x for x in re.findall(r"[^0-9]|[0-9]+", var)
        ],
    )
    
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/{target}/{case_id}_{mri_type}.mat"):
        img3d = np.around(load_nifti_image(nii_path)).astype(np.int16)
        #print(np.min(img3d), np.max(img3d))

        """
        middle = len(files) // 2
        img_half_size = img_size // 2
        p1 = max(0, middle - img_half_size)
        p2 = min(len(files), middle + img_half_size)
        _, slice_thickness = load_dicom_image(files[0], rotate=rotate, get_slice_thickness=True)
        image_stack = [load_dicom_image(f, rotate=rotate).astype(np.int) for f in files[p1:p2]]
        img3d = np.stack(image_stack).T

        print("Before:")
        print(img3d.shape)  
        plt.imshow(img3d[img3d.shape[0]//2,:,:], cmap=cm.Greys_r)
        plt.show()

        plt.imshow(img3d[:,img3d.shape[1]//2,:], cmap=cm.Greys_r)
        plt.show()

        plt.imshow(img3d[:,:,img3d.shape[2]//2], cmap=cm.Greys_r)
        plt.show()


        img3d = [cv2.rotate(cv2.resize(img3d[i,:,:].astype('float32'), (int(img3d.shape[2]*slice_thickness), int(img3d.shape[1]))), cv2.ROTATE_90_CLOCKWISE) for i in range(img3d.shape[0])]
        #img3d = [cv2.resize(img3d[i,:,:].astype('float32'), (int(img3d.shape[2]*slice_thickness), img3d.shape[1])) for i in range(img3d.shape[0])]
        img3d = np.stack(img3d,axis=-1).T

        print(img3d.shape)  

        diff = (img3d.shape[-1]-img_size)/2
        if img3d.shape[-1] < img_size:
            n_zero_bef = np.ones((img_size, img_size, floor(-diff)))*np.min(img3d)
            n_zero_aft = np.ones((img_size, img_size, ceil(-diff)))*np.min(img3d)
            img3d = np.concatenate((n_zero_bef, img3d), axis=-1)
            img3d = np.concatenate((img3d, n_zero_aft), axis=-1)
        elif img3d.shape[-1] > img_size:
            img3d = img3d[:,:,floor(diff):-ceil(diff)]

        img3d = img3d.astype(np.int)
        """

        #print("After:")
        #print(img3d.shape) 
        #plt.imshow(img3d[img3d.shape[0]//2,:,:], cmap=cm.Greys_r)
        #plt.show()

        #plt.imshow(img3d[:,img3d.shape[1]//2,:], cmap=cm.Greys_r)
        #plt.show()

        #plt.imshow(img3d[:,:,img3d.shape[2]//2], cmap=cm.Greys_r)
        #plt.show()


        #img3d = img3d[:img_size,:img_size,:img_size]

        #if np.min(img3d) < np.max(img3d):
        #    img3d = img3d - np.min(img3d)
        #    img3d = img3d / np.max(img3d)

        data = {'X': img3d, 'y': target}
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat")
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/{target}"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/{target}")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/{target}/{case_id}_{mri_type}.mat"):
            io.savemat(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/{target}/{case_id}_{mri_type}.mat", data, do_compression=True)
    #return np.expand_dims(img3d, 0)

In [5]:
def save_brats18_images_3d_as_mat(
    folder,
    mri_type,
    case_id,
    img_size=IMAGE_SIZE
):

    t = ""
    if mri_type == "FLAIR":
        t = "flair"
    elif mri_type == "T1w":
        t = "t1"
    elif mri_type == "T1wCE":
        t = "t1ce"
    elif mri_type == "T2w":
        t = "t2"
    nii_path = f"../../RSNA-BTC-Datasets/{folder}/{mri_type}/{case_id}_{t}.nii"
    
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/1/{case_id}_{mri_type}.mat"):
        img3d = np.around(load_nifti_image(nii_path)).astype(np.int16)
        
        data = {'X': img3d, 'y': 1}
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat")
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/1"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/1")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/1/{case_id}_{mri_type}.mat"):
            io.savemat(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/1/{case_id}_{mri_type}.mat", data, do_compression=True)

In [6]:
def save_openneuro_images_3d_as_mat(
    folder,
    mri_type,
    case_id,
    img_size=IMAGE_SIZE
):

    nii_path = f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{case_id}_{mri_type}.nii.gz"
    
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat"):
        img3d = np.around(load_nifti_image(nii_path)).astype(np.int16)
        
        data = {'X': img3d, 'y': 1}
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat")
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat"):
            io.savemat(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat", data, do_compression=True)

In [7]:
def save_upenn_gbm_images_3d_as_mat(
    folder,
    mri_type,
    case_id,
    img_size=IMAGE_SIZE
):
    nii_path = f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{case_id}_{mri_type}.nii.gz"
    if not os.path.exists(nii_path):
        f_nii_path = f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{case_id[:-3]}_{mri_type}.nii.gz"
        os.rename(f_nii_path, nii_path)
    
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat"):
        img3d = np.around(load_nifti_image(nii_path)).astype(np.int16)
        
        data = {'X': img3d, 'y': 1}
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat")
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0"):
            os.makedirs(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0")  
        if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat"):
            io.savemat(f"../../RSNA-BTC-Datasets/{folder}_mat/{mri_type}/0/{case_id}_{mri_type}.mat", data, do_compression=True)

In [6]:
data = {}
data["train"] = pd.read_csv("../train_labels.csv")
data["test"] = pd.read_csv("../sample_submission.csv")

In [ ]:
missing = {}
missing["train"] = []
missing["test"] = []
for folder,value in data.items():
    for index in tqdm(range(len(value))):
        row = value.loc[index]
        case_id = int(row.BraTS21ID)
        target = int(row.MGMT_value)
        case_id_str = str(case_id).zfill(5)
        if folder == "test":
            target_str = "unknown"
        else:
            target_str = target
        if os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_nii/FLAIR/{target_str}/{case_id_str}.nii.gz"):
            for mri_type in ["FLAIR", "T1w", "T1wCE", "T2w"]:
                save_dicom_images_3d_as_mat(folder,mri_type,case_id,target)
        else:
            missing[folder].append(case_id_str)
                
print("Missing train ids:")
print(missing["train"])
print("Missing test ids:")
print(missing["test"])

In [22]:
folder = "brats18"
for elem in tqdm(os.listdir(f"../../RSNA-BTC-Datasets/{folder}/FLAIR")):
    case_id = elem[:-4]
    case_id = case_id.replace("_flair","").replace("_t1","").replace("_t1ce","").replace("_t2","")
    #print(case_id)
    for mri_type in ["FLAIR", "T1w", "T1wCE", "T2w"]:
        save_brats18_images_3d_as_mat(folder,mri_type,case_id)

100%|█████████████████████████████████████████| 285/285 [16:03<00:00,  3.38s/it]


In [55]:
brats18_data = []
folder = "brats18"
for elem in os.listdir(f"../../RSNA-BTC-Datasets/{folder}_mat/FLAIR/1"):
    case_id = elem[:-4]
    case_id = case_id.replace("_FLAIR","").replace("_T1w","").replace("_T1wCE","").replace("_T2w","")
    brats18_data.append(case_id)
    
print(brats18_data)

brats18_dict = {}
df = pd.DataFrame(columns=["BraTS21ID","MGMT_value"])
for val in brats18_data:
    brats18_dict["BraTS21ID"] = val
    brats18_dict["MGMT_value"] = 1
    df = df.append(brats18_dict, ignore_index=True)
    
df.to_csv("../ext_test_set_1.csv", index=False)

['Brats18_CBICA_AQN_1', 'Brats18_CBICA_AUR_1', 'Brats18_CBICA_ATD_1', 'Brats18_CBICA_BHK_1', 'Brats18_TCIA13_621_1', 'Brats18_CBICA_ALU_1', 'Brats18_CBICA_ATX_1', 'Brats18_TCIA10_393_1', 'Brats18_TCIA06_332_1', 'Brats18_2013_18_1', 'Brats18_2013_22_1', 'Brats18_TCIA06_603_1', 'Brats18_2013_0_1', 'Brats18_TCIA08_167_1', 'Brats18_TCIA06_165_1', 'Brats18_TCIA09_402_1', 'Brats18_TCIA10_644_1', 'Brats18_TCIA12_480_1', 'Brats18_TCIA13_624_1', 'Brats18_TCIA02_117_1', 'Brats18_CBICA_AUQ_1', 'Brats18_TCIA05_444_1', 'Brats18_TCIA08_105_1', 'Brats18_TCIA04_479_1', 'Brats18_CBICA_APR_1', 'Brats18_TCIA08_469_1', 'Brats18_TCIA04_111_1', 'Brats18_TCIA09_462_1', 'Brats18_CBICA_AXW_1', 'Brats18_CBICA_AQR_1', 'Brats18_CBICA_AVJ_1', 'Brats18_TCIA10_261_1', 'Brats18_CBICA_ASE_1', 'Brats18_TCIA02_394_1', 'Brats18_TCIA01_147_1', 'Brats18_TCIA01_186_1', 'Brats18_TCIA10_330_1', 'Brats18_TCIA08_278_1', 'Brats18_TCIA13_653_1', 'Brats18_CBICA_AYI_1', 'Brats18_TCIA10_282_1', 'Brats18_TCIA06_184_1', 'Brats18_CBICA

In [7]:
#folder = "OpenNeuroDS000221"
folder = "OpenNeuroDS000221_ss"
for elem in tqdm(os.listdir(f"../../RSNA-BTC-Datasets/{folder}/0")):
    case_id = elem
    for file in os.listdir(f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}"):
        new_file = file.split("_")
        new_file = f"{new_file[0]}_{new_file[-1]}"
        os.rename(f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{file}", f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{new_file}")
    for mri_type in ["FLAIR", "T1w"]:#, "T1wCE", "T2w"]:
        save_openneuro_images_3d_as_mat(folder,mri_type,case_id)

100%|█████████████████████████████████████████| 231/231 [07:33<00:00,  1.96s/it]


In [56]:
openeuro_data = []
folder = "OpenNeuroDS000221"
for elem in os.listdir(f"../../RSNA-BTC-Datasets/{folder}_mat/FLAIR/0"):
    case_id = elem[:-4]
    case_id = case_id.replace("_FLAIR","").replace("_T1w","").replace("_T1wCE","").replace("_T2w","")
    openeuro_data.append(case_id)
    
print(openeuro_data)

openneuro_dict = {}
df = pd.DataFrame(columns=["BraTS21ID","MGMT_value"])

for val in openeuro_data:
    openneuro_dict["BraTS21ID"] = val
    openneuro_dict["MGMT_value"] = 0
    df = df.append(openneuro_dict, ignore_index=True)
    
df.to_csv("../ext_test_set_0.csv", index=False)

['sub-010121', 'sub-010036', 'sub-010004', 'sub-010145', 'sub-010182', 'sub-010102', 'sub-010018', 'sub-010129', 'sub-010138', 'sub-010065', 'sub-010047', 'sub-010085', 'sub-010228', 'sub-010011', 'sub-010232', 'sub-010188', 'sub-010140', 'sub-010234', 'sub-010010', 'sub-010230', 'sub-010049', 'sub-010152', 'sub-010205', 'sub-010107', 'sub-010097', 'sub-010212', 'sub-010176', 'sub-010210', 'sub-010112', 'sub-010038', 'sub-010150', 'sub-010206', 'sub-010167', 'sub-010189', 'sub-010143', 'sub-010022', 'sub-010061', 'sub-010238', 'sub-010209', 'sub-010069', 'sub-010168', 'sub-010117', 'sub-010076', 'sub-010204', 'sub-010233', 'sub-010074', 'sub-010181', 'sub-010186', 'sub-010016', 'sub-010193', 'sub-010059', 'sub-010163', 'sub-010195', 'sub-010012', 'sub-010099', 'sub-010017', 'sub-010135', 'sub-010111', 'sub-010165', 'sub-010151', 'sub-010084', 'sub-010098', 'sub-010136', 'sub-010197', 'sub-010159', 'sub-010123', 'sub-010043', 'sub-010042', 'sub-010246', 'sub-010241', 'sub-010001', 'sub-

In [8]:
#folder = "OpenNeuroDS000221"
folder = "UPENN-GBM"
for elem in tqdm(os.listdir(f"../../RSNA-BTC-Datasets/{folder}/0")):
    case_id = elem
    for file in os.listdir(f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}"):
        new_file = file.split("_")
        new_file = f"{new_file[0]}_{new_file[-1]}"
        os.rename(f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{file}", f"../../RSNA-BTC-Datasets/{folder}/0/{case_id}/{new_file}")
    for mri_type in ["FLAIR", "T1", "T1GD", "T2"]:#, "T1wCE", "T2w"]:
        save_upenn_gbm_images_3d_as_mat(folder,mri_type,case_id)

100%|█████████████████████████████████████████| 671/671 [09:17<00:00,  1.20it/s]


In [9]:
folder = "UPENN-GBM_mat"
for seq in os.listdir(f"../../RSNA-BTC-Datasets/{folder}"):
    for elem in tqdm(os.listdir(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0")):
        if seq == "T1":
            os.rename(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}", f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}"[:-4]+"w.mat")
        elif seq == "T2":
            os.rename(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}", f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}"[:-4]+"w.mat")
        elif seq == "T1GD":
            os.rename(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}", f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}"[:-6]+"wCE.mat")

100%|██████████████████████████████████████| 671/671 [00:00<00:00, 79956.19it/s]


In [10]:
upenngbm_data = pd.read_csv("../UPENN-GBM_clinical_info_v1.1.csv")
print(upenngbm_data)


                     ID Gender  Age_at_scan_years Survival_from_surgery_days  \
0    UPENN-GBM-00001_11      F              52.16              Not Available   
1    UPENN-GBM-00002_11      F              61.30                        291   
2    UPENN-GBM-00003_11      M              42.82              Not Available   
3    UPENN-GBM-00004_11      M              33.43              Not Available   
4    UPENN-GBM-00005_11      M              53.33              Not Available   
..                  ...    ...                ...                        ...   
666  UPENN-GBM-00626_21      F              57.19              Not Available   
667  UPENN-GBM-00627_21      F              57.40              Not Available   
668  UPENN-GBM-00628_21      M              55.09              Not Available   
669  UPENN-GBM-00629_21      M              71.44              Not Available   
670  UPENN-GBM-00630_21      M              65.47              Not Available   

         IDH1           MGMT           

In [11]:
upenngbm_filtered_data = upenngbm_data[((upenngbm_data["MGMT"] == "Methylated") | (upenngbm_data["MGMT"] == "Unmethylated"))]
print(upenngbm_filtered_data)

                     ID Gender  Age_at_scan_years Survival_from_surgery_days  \
21   UPENN-GBM-00022_11      F              53.88                       1882   
33   UPENN-GBM-00034_11      F              53.63                        464   
95   UPENN-GBM-00088_11      M              47.32                        334   
99   UPENN-GBM-00091_11      M              70.54                        200   
100  UPENN-GBM-00092_11      F              66.44              Not Available   
..                  ...    ...                ...                        ...   
664  UPENN-GBM-00624_21      M              44.33              Not Available   
665  UPENN-GBM-00625_21      F              48.75              Not Available   
666  UPENN-GBM-00626_21      F              57.19              Not Available   
669  UPENN-GBM-00629_21      M              71.44              Not Available   
670  UPENN-GBM-00630_21      M              65.47              Not Available   

         IDH1          MGMT            

In [12]:
methylated = upenngbm_filtered_data[upenngbm_filtered_data["MGMT"] == "Methylated"]
unmethylated = upenngbm_filtered_data[upenngbm_filtered_data["MGMT"] == "Unmethylated"]
m_list = list(methylated["ID"])
u_list = list(unmethylated["ID"])
print(m_list)
print(u_list)

['UPENN-GBM-00095_11', 'UPENN-GBM-00098_11', 'UPENN-GBM-00115_11', 'UPENN-GBM-00124_11', 'UPENN-GBM-00128_11', 'UPENN-GBM-00128_21', 'UPENN-GBM-00130_21', 'UPENN-GBM-00131_11', 'UPENN-GBM-00135_11', 'UPENN-GBM-00138_11', 'UPENN-GBM-00183_21', 'UPENN-GBM-00197_11', 'UPENN-GBM-00197_21', 'UPENN-GBM-00240_11', 'UPENN-GBM-00260_11', 'UPENN-GBM-00263_11', 'UPENN-GBM-00264_11', 'UPENN-GBM-00269_11', 'UPENN-GBM-00287_11', 'UPENN-GBM-00302_11', 'UPENN-GBM-00310_11', 'UPENN-GBM-00344_11', 'UPENN-GBM-00344_21', 'UPENN-GBM-00350_11', 'UPENN-GBM-00351_11', 'UPENN-GBM-00352_21', 'UPENN-GBM-00356_11', 'UPENN-GBM-00358_11', 'UPENN-GBM-00359_11', 'UPENN-GBM-00360_11', 'UPENN-GBM-00363_11', 'UPENN-GBM-00371_11', 'UPENN-GBM-00377_11', 'UPENN-GBM-00378_11', 'UPENN-GBM-00380_11', 'UPENN-GBM-00381_11', 'UPENN-GBM-00388_11', 'UPENN-GBM-00392_11', 'UPENN-GBM-00395_11', 'UPENN-GBM-00399_11', 'UPENN-GBM-00401_11', 'UPENN-GBM-00403_11', 'UPENN-GBM-00404_11', 'UPENN-GBM-00406_11', 'UPENN-GBM-00407_11', 'UPENN-GB

In [20]:
folder = "UPENN-GBM_mat"
if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_useless"):
    os.mkdir(f"../../RSNA-BTC-Datasets/{folder}_useless")
for seq in os.listdir(f"../../RSNA-BTC-Datasets/{folder}"):
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_useless/{seq}"):
        os.mkdir(f"../../RSNA-BTC-Datasets/{folder}_useless/{seq}")
    if not os.path.exists(f"../../RSNA-BTC-Datasets/{folder}_useless/{seq}/0"):
        os.mkdir(f"../../RSNA-BTC-Datasets/{folder}_useless/{seq}/0")
    for elem in tqdm(os.listdir(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0")):
        el = elem.replace("_FLAIR","").replace("_T1w","").replace("_T1wCE","").replace("_T2w","").replace(".mat","").replace("1CE","1")
        print(el)
        if el in m_list:
            print("m")
            os.rename(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}",f"../../RSNA-BTC-Datasets/{folder}/{seq}/1/{elem}")
        elif el in u_list:
            print("u")
        else:
            print("useless")
            os.rename(f"../../RSNA-BTC-Datasets/{folder}/{seq}/0/{elem}",f"../../RSNA-BTC-Datasets/{folder}_useless/{seq}/0/{elem}")

100%|██████████████████████████████████████| 170/170 [00:00<00:00, 20649.03it/s]


UPENN-GBM-00133_11
u
UPENN-GBM-00120_11
u
UPENN-GBM-00267_11
u
UPENN-GBM-00482_11
u
UPENN-GBM-00375_11
u
UPENN-GBM-00244_11
u
UPENN-GBM-00405_11
u
UPENN-GBM-00463_11
u
UPENN-GBM-00134_21
u
UPENN-GBM-00370_11
u
UPENN-GBM-00492_11
u
UPENN-GBM-00093_21
u
UPENN-GBM-00584_11
u
UPENN-GBM-00290_11
u
UPENN-GBM-00267_21
u
UPENN-GBM-00462_11
u
UPENN-GBM-00480_11
u
UPENN-GBM-00581_11
u
UPENN-GBM-00448_11
u
UPENN-GBM-00140_11
u
UPENN-GBM-00585_11
u
UPENN-GBM-00483_11
u
UPENN-GBM-00034_11
u
UPENN-GBM-00092_11
u
UPENN-GBM-00563_11
u
UPENN-GBM-00093_11
u
UPENN-GBM-00562_11
u
UPENN-GBM-00508_11
u
UPENN-GBM-00385_11
u
UPENN-GBM-00625_21
u
UPENN-GBM-00246_11
u
UPENN-GBM-00452_11
u
UPENN-GBM-00457_11
u
UPENN-GBM-00470_11
u
UPENN-GBM-00369_11
u
UPENN-GBM-00384_11
u
UPENN-GBM-00132_11
u
UPENN-GBM-00486_11
u
UPENN-GBM-00428_11
u
UPENN-GBM-00471_11
u
UPENN-GBM-00575_11
u
UPENN-GBM-00426_11
u
UPENN-GBM-00596_11
u
UPENN-GBM-00549_11
u
UPENN-GBM-00282_11
u
UPENN-GBM-00393_11
u
UPENN-GBM-00312_11
u
UPENN-GBM-001

100%|███████████████████████████████████████| 550/550 [00:00<00:00, 7448.16it/s]


UPENN-GBM-00120_11
u
UPENN-GBM-00295_11
useless
UPENN-GBM-00281_11
useless
UPENN-GBM-00148_11
u
UPENN-GBM-00561_11
useless
UPENN-GBM-00347_11
useless
UPENN-GBM-00410_11
u
UPENN-GBM-00156_11
useless
UPENN-GBM-00431_11
u
UPENN-GBM-00508_11
u
UPENN-GBM-00154_11
useless
UPENN-GBM-00307_11
useless
UPENN-GBM-00332_21
useless
UPENN-GBM-00465_11
useless
UPENN-GBM-00052_21
useless
UPENN-GBM-00601_11
useless
UPENN-GBM-00396_11
useless
UPENN-GBM-00291_11
useless
UPENN-GBM-00323_11
useless
UPENN-GBM-00483_11
u
UPENN-GBM-00227_11
useless
UPENN-GBM-00110_11
useless
UPENN-GBM-00398_11
u
UPENN-GBM-00049_11
useless
UPENN-GBM-00228_11
useless
UPENN-GBM-00219_11
useless
UPENN-GBM-00549_11
u
UPENN-GBM-00581_11
u
UPENN-GBM-00192_11
useless
UPENN-GBM-00209_11
useless
UPENN-GBM-00616_21
u
UPENN-GBM-00142_11
useless
UPENN-GBM-00181_11
useless
UPENN-GBM-00328_11
useless
UPENN-GBM-00183_11
useless
UPENN-GBM-00065_11
useless
UPENN-GBM-00312_21
u
UPENN-GBM-00397_11
useless
UPENN-GBM-00119_11
useless
UPENN-GBM-003

100%|██████████████████████████████████████| 170/170 [00:00<00:00, 15433.92it/s]


UPENN-GBM-00484_11
u
UPENN-GBM-00593_11
u
UPENN-GBM-00549_11
u
UPENN-GBM-00336_11
u
UPENN-GBM-00093_11
u
UPENN-GBM-00384_11
u
UPENN-GBM-00591_11
u
UPENN-GBM-00463_11
u
UPENN-GBM-00435_11
u
UPENN-GBM-00370_11
u
UPENN-GBM-00454_11
u
UPENN-GBM-00500_11
u
UPENN-GBM-00470_11
u
UPENN-GBM-00385_11
u
UPENN-GBM-00034_11
u
UPENN-GBM-00508_11
u
UPENN-GBM-00143_11
u
UPENN-GBM-00391_11
u
UPENN-GBM-00629_21
u
UPENN-GBM-00527_11
u
UPENN-GBM-00379_11
u
UPENN-GBM-00531_11
u
UPENN-GBM-00616_21
u
UPENN-GBM-00093_21
u
UPENN-GBM-00587_11
u
UPENN-GBM-00458_11
u
UPENN-GBM-00596_11
u
UPENN-GBM-00457_11
u
UPENN-GBM-00355_21
u
UPENN-GBM-00604_11
u
UPENN-GBM-00552_11
u
UPENN-GBM-00459_11
u
UPENN-GBM-00290_11
u
UPENN-GBM-00499_11
u
UPENN-GBM-00244_11
u
UPENN-GBM-00626_21
u
UPENN-GBM-00092_11
u
UPENN-GBM-00282_11
u
UPENN-GBM-00267_11
u
UPENN-GBM-00492_11
u
UPENN-GBM-00431_11
u
UPENN-GBM-00151_11
u
UPENN-GBM-00574_11
u
UPENN-GBM-00533_11
u
UPENN-GBM-00134_11
u
UPENN-GBM-00314_21
u
UPENN-GBM-00432_11
u
UPENN-GBM-003

100%|██████████████████████████████████████| 170/170 [00:00<00:00, 26250.11it/s]


UPENN-GBM-00120_11
u
UPENN-GBM-00436_11
u
UPENN-GBM-00570_11
u
UPENN-GBM-00482_11
u
UPENN-GBM-00384_11
u
UPENN-GBM-00352_11
u
UPENN-GBM-00134_21
u
UPENN-GBM-00448_11
u
UPENN-GBM-00267_11
u
UPENN-GBM-00316_11
u
UPENN-GBM-00133_11
u
UPENN-GBM-00088_11
u
UPENN-GBM-00459_11
u
UPENN-GBM-00355_21
u
UPENN-GBM-00434_11
u
UPENN-GBM-00391_11
u
UPENN-GBM-00597_11
u
UPENN-GBM-00464_11
u
UPENN-GBM-00375_11
u
UPENN-GBM-00362_11
u
UPENN-GBM-00508_11
u
UPENN-GBM-00034_11
u
UPENN-GBM-00151_11
u
UPENN-GBM-00558_11
u
UPENN-GBM-00150_11
u
UPENN-GBM-00148_11
u
UPENN-GBM-00368_11
u
UPENN-GBM-00244_11
u
UPENN-GBM-00449_11
u
UPENN-GBM-00416_11
u
UPENN-GBM-00591_11
u
UPENN-GBM-00596_11
u
UPENN-GBM-00489_11
u
UPENN-GBM-00376_11
u
UPENN-GBM-00398_11
u
UPENN-GBM-00590_11
u
UPENN-GBM-00581_11
u
UPENN-GBM-00428_11
u
UPENN-GBM-00490_11
u
UPENN-GBM-00148_21
u
UPENN-GBM-00093_11
u
UPENN-GBM-00421_11
u
UPENN-GBM-00125_11
u
UPENN-GBM-00139_11
u
UPENN-GBM-00560_11
u
UPENN-GBM-00605_11
u
UPENN-GBM-00367_11
u
UPENN-GBM-004

0it [00:00, ?it/s]
